In [8]:
import sys 
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import random 
import math
!pip install opencv-python

In [9]:
cap = cv2.VideoCapture(0)


if not cap.isOpened():

    print('Camera open failed!')

    sys.exit()

먼저 기본 카메라의 Width와 Height을 받아와야겠죠

In [10]:
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

연산속도를 빠르게 하기위해 반으로 리사이즈 해줍니다.

In [14]:
w2 = w // 2

h2 = h // 2

첫번째 프레임은 이전프레임으로 사용할 것입니다. 그리고 두번째 프레임은 세번째 프레임의 이전프레임이 되겠죠.

In [11]:
ret, frame = cap.read()

if not ret:

    print('Frame read failed!')

    sys.exit()

기본적인 카메라는 반대방향이기에 거울보듯이 움직이게끔 하기위해 flip을 사용합니다.

In [12]:
frame = cv2.flip(frame, 1)  # 좌우 대칭

gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

gray1 = cv2.resize(gray1, (w2, h2), interpolation=cv2.INTER_AREA)

두번째 프레임부터 계산을 합니다. gray1이 이전프레임이며, gray2는 현재 프레임이겠죠.

In [13]:
while True:

    ret, frame = cap.read()

    #두번째 프레임을 받아온다.

    if not ret:

        print('Frame read failed!')

        break


    frame = cv2.flip(frame, 1)  # 좌우 대칭

    gray2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    gray2 = cv2.resize(gray2, (w2, h2), interpolation=cv2.INTER_AREA)

    #똑같이 그레이스케일로변환하고 리사이즈 처리한다.


    # 밀집 옵티컬플로우 계산

    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.1, 0)

    #gray1이 이전프레임, gray2가 현재프레임. 전체픽셀의 OpticalFlow를 계산한다.  여기서 flow는 (h2,w2,2)의 shape으로 저장이 된다. 첫번째프레임과 두번째 프레임 방향을 직각좌표계 성분으로 리턴



    vx, vy = flow[..., 0], flow[..., 1]

    mag, ang = cv2.cartToPolar(vx, vy)

    #변수이름 따로 저장 magnitude, angle

    #직각좌표계 성분을 극좌표계 성분으로 바꾸어준다.

    #그러면 벡터의 크기와 각도성분으로 계산이 된다.

    



    # 움직임 벡터 시각화

    hsv = np.zeros((h2, w2, 3), dtype=np.uint8) #일단 검은색 화면으로 받음



    hsv[..., 0] = ang*180/np.pi/2 #Hue값은 각도니, 각도로 색깔 표현



    hsv[..., 1] = 255 #saturation은 255로 고정. 항상 선명하게



    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)#value는 Magnitude로 본다. 255일수록 밝아진다.



    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    #그리고 BGR로 HSV를 변환해서 화면에 출력한다.

    cv2.imshow('flow', bgr)




    # 움직임이 충분히 큰 영역 선택 '모션마스크만들기'

    motion_mask = np.zeros((h2, w2), dtype=np.uint8)

    motion_mask[mag > 2.0] = 255#->이건 노이즈 감지 무시 목적

    #노이즈를 방지하기 위해서 2픽셀정도 움직임만 움직임 모션마스크를 만든다. 움직임이 있으면 흰색, 없으면 검은색 마스크생성



    mx = cv2.mean(vx, mask=motion_mask)[0]#x방향성분의 움직임벡터

    my = cv2.mean(vy, mask=motion_mask)[0]#y방향성분의 움직임벡터의 평균 계산.

# double값 네개를 리턴하므로 마지막의 [0]을해줘야 첫번째 그레이스케일 영상을 저장해줄 수 있다.

    m_mag = math.sqrt(mx*mx + my*my)

    #두개를 이용해 magnitude를 다시 계산해준다.



    if m_mag > 4.0:

 #평균벡터의 크기가 4이상이면 모션벡터를 감지한다. 즉 2픽셀 이상 움직인것만 평균을 계산하고 실제 크기가 4이상이어야 다룬다. -> 이건 충분히 움직임을 클때만 동작시키는 조건

        m_ang = math.atan2(my, mx) * 180 / math.pi

        #평균모션벡터의 방향성분을 계산한다. arctan2함수를 이용하면 라디안에서 -180~+180으로 변환이 된다.

        m_ang += 180

        #이값이 -pi~+pi

        #여기서 m_ang이 0이면 손을 왼쪽으로 인식한것이다.

        #위는 90도 오른쪽은 180 아래는 270도이다.


        pt1 = (100, 100)



        if m_ang >= 45 and m_ang < 135:

            pt2 = (100, 30)

            #평균방향이 45~135인 위이면, 화살표의 끝점으로 그린다.

        elif m_ang >= 135 and m_ang < 225:

            pt2 = (170, 100)

            #X좌표 70증가

        elif m_ang >= 225 and m_ang < 315:

            pt2 = (100, 170)

            #Y좌표 70증가

        else:

            pt2 = (30, 100)



        cv2.arrowedLine(frame, pt1, pt2, (0, 0, 255),

                        7, cv2.LINE_AA, tipLength=0.7)

        #화살표그려주는 함수

    # 결과 영상 화면 출력

    cv2.imshow('frame', frame)

    cv2.imshow('motion_mask', motion_mask)

    #모션마스크

    if cv2.waitKey(1) == 27:

        break


    # 현재 프레임을 이전 프레임으로 복사

    gray1 = gray2



cap.release()

cv2.destroyAllWindows()